## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


User-User - сравнение похожих пользователей. Рекомендация пользователю товаров купленных похожими на него пользователями. 

Item-Item - сравнение похожих товаров. Рекомендация пользователю товаров похожих на купленные им товары.

Различия у данных рекомендаций заключается в подходах (поиск похожих пользователей и поиск похожих товаров). Похожи тем, что похожесть пользователей/товаров определяется из взаимодействия пользователей с товарами (user-item interactions). 

Функцию item_item_rec можно использовать для user_user_rec, если предварительно транспонировать interaction_matrix.



В item-item используется осуществляется поиск похожих пользователей/товаров, на основании похожести товаров/пользователей делаются рекомендации. 
В ALS используется матричное разложение. При раскладывании на компоненты удается сжать исходную информацию таким образом, что при перемножении матриц получается близкая к исходной матрице интеракций. В полученной матрице отсутствующие значения (отсутствие взаимодействия пользователя с товаром) заполняются некоторыми значениями. Данные значения **после нормирования по товарам** можно интерпретировать как вероятность релевантности рекомендации товаров пользователям.

___

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


* Маржинальность товаров. Более высокие веса для товаров с большей маржинальностью;
* Сезонность. Более высокие веса для товаров характерные для текущего сезона. Шапки, шарфы и т.д. - зимой. Майки, шорты и т.д. - летом;
* Акционные товары. С большей вероятностью будет куплен товар, на который действует скидка.

___

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


В качестве признаков используются только взаимодействия пользователей с товарами. Не учитываются такие признаки, как: пол, возраст, наличие/отсутствие детей и т.д.;

Высокая вычислительная сложность при большом количестве факторов;

Необходимы факты взаимодействия пользователей и товаров. Каждый пользователь должен сделать хотя бы одну покупку и каждый товар должен быть куплен хотя бы один раз. Проблема холодного старта.

___

## Практическая часть


In [2]:
# !pip install implicit

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import os, sys

import time
from itertools import product
from tqdm import tqdm

In [2]:
path = './data/retail_train.csv'

data = pd.read_csv(path)

In [3]:
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


### Оптимизация гиперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [12]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [11]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [71]:
weights = {
    '': None, 
    'tfidf': tfidf_weight, 
    'bm25': bm25_weight
}
n_factors = [10, 50, 100, 200]
regs = [1e-3, 1e-2, 1e-1, 1.]

In [72]:
temp = data_test.groupby('user_id')['item_id'].unique().reset_index()
temp.columns=['user_id', 'actual']

results = []


for weight in weights.keys():
    if weights[weight] is not None:
        interactions = weights[weight](user_item_matrix.T).T
    else:
        interactions = user_item_matrix.copy()
        
    for factors, regularization in tqdm(product(n_factors, regs)):

        model = AlternatingLeastSquares(factors=factors, 
                                        regularization=regularization,
                                        iterations=15, 
                                        num_threads=8)

        start_time = time.time()
        model.fit(csr_matrix(interactions).T.tocsr(), show_progress=False)
        duration = round(time.time() - start_time)

        temp['recommended'] = temp['user_id'].apply(lambda x: get_recommendations(x, model, N=5))

        score = temp.apply(lambda row: precision_at_k(row['recommended'], row['actual']), axis=1).mean()

        results.append([weight, factors, regularization, duration, score])
        
        
results = pd.DataFrame(results, columns=['weight', 'n_factors', 'regularization', 'training_time', 'score'])

16it [11:49, 44.36s/it]
16it [14:51, 55.73s/it]
16it [13:45, 51.58s/it]


In [73]:
results

,weight,n_factors,regularization,training_time,score
0,,10,0.001,2,0.191381
1,,10,0.010,2,0.189520
2,,10,0.100,2,0.194613
3,,10,1.000,2,0.188737
4,,50,0.001,4,0.182958
5,,50,0.010,4,0.185896
6,,50,0.100,4,0.185798
7,,50,1.000,4,0.184035
8,,100,0.001,4,0.182370
9,,100,0.010,5,0.183643


In [81]:
results.sort_values(by='score', ascending=False)

,weight,n_factors,regularization,training_time,score
46,bm25,200,0.100,7,0.237904
45,bm25,200,0.010,7,0.235162
47,bm25,200,1.000,8,0.232321
44,bm25,200,0.001,8,0.230852
42,bm25,100,0.100,5,0.225367
41,bm25,100,0.010,6,0.225171
43,bm25,100,1.000,5,0.223604
40,bm25,100,0.001,7,0.219687
37,bm25,50,0.010,6,0.204897
17,tfidf,10,0.010,3,0.203134


BM25 взвешивание показало себя наилучшим образом.

Гиперпараметры лучшей модели:

* factors = 200;
* regularization = 0.1

Интересно здесь то, что без взвешивания и при TF-IDF взвешивании метрика оказалась выше у моделей с меньшим количеством факторов. При BM25 напротив, больше факторов - лучше результат.